In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import warnings 
warnings.filterwarnings("ignore")

# load crime data
crimes = pd.read_csv('Crimes_-_2001_to_present.csv', memory_map =True)

#Get how many NaN value we have and where the values are
crimes.isna().sum()

#Drop NaN values
crimes.dropna(axis=0,inplace=True)

#Drop unneccessary columns
crimes.drop(["IUCR","FBI Code","X Coordinate","Y Coordinate"],axis=1,inplace=True)

In [5]:
#load community area data
com = pd.read_csv('ChicagoComName.csv', memory_map =True)

#Drop unneccessary columns
com.drop(['latinos','blacks','white','asian','other','Sides','income'], axis=1,inplace=True)

com.head() 

,Community Area,Community Name,population
0,1,Rogers Park,54991
1,2,West Ridge,71942
2,3,Uptown,56362
3,4,Lincoln Square,39493
4,5,North Center,31867


In [6]:
# convert dates to pandas datetime format
crimes.Date = pd.to_datetime(crimes.Date, format='%m/%d/%Y %I:%M:%S %p')
# setting the index to be the date will help us a lot later on
crimes.index = pd.DatetimeIndex(crimes.Date)

# of records X # of features
crimes.shape

crimes.head()

,ID,Case Number,Date,Block,Primary Type,Description,Location Description,Arrest,Domestic,Beat,...,Longitude,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards,Boundaries - ZIP Codes,Police Districts,Police Beats
Date,,,,,,,,,,,,,,,,,,,,,
2019-07-17 23:55:00,11761790,JC353547,2019-07-17 23:55:00,002XX S STATE ST,BATTERY,AGG: HANDS/FIST/FEET NO/MINOR INJURY,CTA PLATFORM,False,False,113.0,...,-87.627691,"(41.878638996, -87.627691486)",22.0,14912.0,38.0,92.0,36.0,29.0,22.0,126.0
2019-07-17 23:55:00,11761782,JC353570,2019-07-17 23:55:00,023XX W TOUHY AVE,THEFT,$500 AND UNDER,VEHICLE NON-COMMERCIAL,False,False,2411.0,...,-87.689264,"(42.012270911, -87.689264024)",42.0,22528.0,20.0,268.0,27.0,12.0,11.0,35.0
2019-07-17 23:55:00,11761756,JC353561,2019-07-17 23:55:00,068XX N SHERIDAN RD,ASSAULT,AGG PO HANDS NO/MIN INJURY,SIDEWALK,True,False,2431.0,...,-87.660994,"(42.005849323, -87.660994398)",3.0,21853.0,10.0,267.0,5.0,9.0,11.0,39.0
2019-07-17 23:50:00,11761796,JC353568,2019-07-17 23:50:00,0000X E 21ST ST,ROBBERY,STRONGARM - NO WEAPON,RESIDENCE,False,False,131.0,...,-87.624784,"(41.854271374, -87.624784323)",48.0,21194.0,34.0,374.0,9.0,40.0,22.0,94.0
2019-07-17 23:47:00,11761761,JC353527,2019-07-17 23:47:00,070XX S CLYDE AVE,ASSAULT,SIMPLE,APARTMENT,False,True,331.0,...,-87.573879,"(41.767089837, -87.573879419)",32.0,22538.0,39.0,152.0,33.0,24.0,18.0,262.0


In [7]:
# Filter out 2016 to 2018 based on finding abve
crimes_2016 = crimes[crimes['Year']==2016]
crimes_2017 = crimes[crimes['Year']==2017]
crimes_2018 = crimes[crimes['Year']==2018]

# Make them pd.dataframe
crimes_2016 = pd.DataFrame(crimes_2016)
crimes_2017 = pd.DataFrame(crimes_2017)
crimes_2018 = pd.DataFrame(crimes_2018)
# Merge crime data and community area data
crimes_2018 = pd.merge(crimes_2018,com,how = 'left', on='Community Area')


In [2]:
# Import folium to plot the map
import folium

MapOfChicago = folium.Map(location=[41.881832, -87.623177],
                         zoom_start=11,tiles = "Stamen Terrain")

com_geo = 'Boundaries - Neighborhoods.geojson'


In [8]:
MapOfChicago.choropleth(
 geo_data=com_geo,
 data=com,
 columns=['Community Name', 'population'],
 key_on='feature.properties.pri_neigh',
 fill_color='YlOrRd',
 fill_opacity=0.7,
 line_opacity=0.2,
 legend_name='population'
)


MapOfChicago.save('folium_chloropleth_Chicago.html')

In [ ]:
crimes_2018 = crimes_2018[crimes_2018['Date'].dt.month.isin([7,8,9,10,11,12])]
crime_in_tour = crimes_2018[crimes_2018["Primary Type"].isin (['THEFT','BATTERY'])]
locations = crime_in_tour.groupby('Community Area').first()
new_locations = locations.loc[:,['Latitude', 'Longitude', 'Location Description', 'Arrest','Community Name']]


In [ ]:
# Plot out two most dangerous type (Theft,battery) in the 2018
for i in range(len(new_locations)):
    lat = new_locations.iloc[i][0]
    long = new_locations.iloc[i][1]
    popup_text = """Community Index : {}<br>
                Arrest : {}<br>
                Location Description : {}<br>
                Community Name: {}<br>"""
    popup_text = popup_text.format(new_locations.index[i],
                               new_locations.iloc[i][-2],
                               new_locations.iloc[i][-3],
                               new_locations.iloc[i][-1]
                               )
    folium.CircleMarker(location = [lat, long], popup= popup_text, fill = True).add_to(MapOfChicago)

#Display the map
MapOfChicago



MapOfChicago.save('Street Threat in second half of 2018.html')

In [174]:
# custom icons

#group 4
fb = folium.features.CustomIcon('http://cdn.onlinewebfonts.com/svg/img_489534.png', icon_size=(50,50))
sc = folium.features.CustomIcon('http://www.sclance.com/pngs/football-icon-png/football_icon_png_524048.png', icon_size=(50,50))
sp = folium.features.CustomIcon('https://cdn4.iconfinder.com/data/icons/building-1/512/build21-512.png', icon_size=(50,50))
mus = folium.features.CustomIcon('https://png.pngtree.com/svg/20160202/musichall_1085227.png', icon_size=(50,50))

#group 3
cg = folium.features.CustomIcon('https://image.flaticon.com/icons/png/512/404/404777.png', icon_size=(50,50))
mu = folium.features.CustomIcon('http://icons.iconarchive.com/icons/iconsmind/outline/256/Museum-icon.png', icon_size=(50,50))
pt = folium.features.CustomIcon('https://png.pngtree.com/svg/20161008/ea9a58058b.svg', icon_size=(50,50))
wt = folium.features.CustomIcon('https://static.thenounproject.com/png/2055041-200.png', icon_size=(50,50))
fw = folium.features.CustomIcon('https://img.icons8.com/cotton/2x/ferris-wheel--v1.png', icon_size=(50,50))

#group 2
ct = folium.features.CustomIcon('https://cdn0.iconfinder.com/data/icons/food-volume-1-3/256/56-512.png', icon_size=(50,50))
kt = folium.features.CustomIcon('https://cdn2.iconfinder.com/data/icons/korea/813/bbq_grill_pork_korea_korean_food-512.png', icon_size=(50,50))
li = folium.features.CustomIcon('https://img.clipartimage.com/medium-image-png-indian-curry-clipart-800_487.png', icon_size=(50,50))
dp = folium.features.CustomIcon('https://vignette.wikia.nocookie.net/clubpenguin/images/a/a9/Puffle_Care_Icons_Pizzapepperoni.png/revision/latest?cb=20131003041210', icon_size=(50,50))

#group 1
ob = folium.features.CustomIcon('https://image.flaticon.com/icons/png/512/63/63611.png', icon_size=(50,50))
ob1 = folium.features.CustomIcon('https://icon-library.net/images/building-icon-png/building-icon-png-19.jpg', icon_size=(50,50))
ob2 = folium.features.CustomIcon('https://i.pinimg.com/originals/15/d9/98/15d998c03f08e570cd70e36f31bdd2d6.png', icon_size=(50,50))
ob3 = folium.features.CustomIcon('https://cdn.iconscout.com/icon/premium/png-256-thumb/office-building-1436127-1212688.png', icon_size=(50,50))

In [175]:
# Define groups
from folium import plugins


mcg = folium.plugins.MarkerCluster(control=False)
MapOfChicago.add_child(mcg)

g1 = folium.plugins.FeatureGroupSubGroup(mcg, 'Work & living')
MapOfChicago.add_child(g1)

g2 = folium.plugins.FeatureGroupSubGroup(mcg, 'Food')
MapOfChicago.add_child(g2)

g3 = folium.plugins.FeatureGroupSubGroup(mcg, 'Tourist Attraction')
MapOfChicago.add_child(g3)

g4 = folium.plugins.FeatureGroupSubGroup(mcg, 'Events')
MapOfChicago.add_child(g4)

# Group 1
points_BD = [[41.8744,-87.6247], [41.8780,-87.6385], [41.8847,-87.6413], [41.8877, -87.6212],[41.8855,-87.6154],[41.8744,-87.6247]]
folium.PolyLine(points_BD, color="green", popup = 'Business District: Avg.Monthly Rent Cost $2,000', weight=3, opacity=1).add_to(g1)
folium.Marker([41.8805,-87.6364], popup='Deloitt Chicago',icon=ob).add_to(g1)
folium.Marker([41.8823,-87.6302], popup='Exelon Corporation Headquarters',icon=ob1).add_to(g1)
folium.Marker([41.8850,-87.6386], popup='Hyatt Hotels Corporation',icon=ob2).add_to(g1)
folium.Marker([41.8849,-87.6363], popup='Ernst & Young',icon=ob3).add_to(g1)

points_Apt = [[41.9179,-87.6349], [41.9180,-87.6587], [41.9247,-87.6677], [41.9277, -87.6409], [41.9179,-87.6349]]
folium.PolyLine(points_Apt, color="yellow", popup = 'Recommned living Area:Avg.Monthly Rent Cost: $1,200', weight=3, opacity=1).add_to(g1)
folium.Marker([41.9213,-87.6380], popup='Margaret Day Blake Apts: $1,000 ',icon=folium.Icon(icon='home',color ='yellow')).add_to(g1)
folium.Marker([41.9260,-87.6400], popup='Beachwalk Apartments: $1,200',icon=folium.Icon(icon='home',color ='yellow')).add_to(g1)

#Group 2
folium.Marker([41.8507,-87.6340], popup='China Town',icon=ct).add_to(g2)
folium.Marker([41.9744,-87.6976], popup='Korean Town',icon=kt).add_to(g2)
folium.Marker([41.9977,-87.6916], popup='Little India',icon=li).add_to(g2)
folium.Marker([41.8969,-87.6259], popup='Giordanos',icon=dp).add_to(g2)

#Group 3
folium.Marker([41.8826,-87.6226], popup='Millennium Park',icon=cg).add_to(g3)
folium.Marker([41.8789,-87.6359], popup='Willis Tower',icon=wt).add_to(g3)
folium.Marker([41.8663,-87.6068], popup='Adler Planetarium',icon=pt).add_to(g3)
folium.Marker([41.8917,-87.6043], popup='Navy Pier',icon=fw).add_to(g3)
folium.Marker([41.8796,-87.6237], popup='The Art Institute of Chicago',icon=mu).add_to(g3)

#Group 4
folium.Marker([41.9751,-87.8680], popup='Fashion Outlets of Chicago',icon=sp).add_to(g4)
folium.Marker([41.7648,-87.8054], popup='SeatGeek Stadium',icon=sc).add_to(g4)
folium.Marker([41.8625,-87.6166], popup='Soldier Field',icon=fb).add_to(g4)
folium.Marker([41.8722,-87.6189], popup='Lollapalooza',icon=mus).add_to(g4)


folium.LayerControl(collapsed=False).add_to(MapOfChicago)

#Display the map
MapOfChicago.save('Safely enjoy Chicago.html')
